In [1]:
import os
from dotenv import load_dotenv
from pydantic import create_model
import inspect, json
from inspect import Parameter
from langchain_community.chat_models import ChatFireworks
from langchain.schema import SystemMessage, HumanMessage

print(load_dotenv())

True


## **Custom agent**

**Define the functions**

In [2]:
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [3]:
abc(2, 3)

-2

In [4]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [5]:
abc_json = jsonschema(abc)
abc_json

/tmp/ipykernel_18014/3911407637.py:13: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'abc',
 'description': 'Compute abc between two numbers',
 'parameters': {'properties': {'num1': {'title': 'Num1', 'type': 'integer'},
   'num2': {'title': 'Num2', 'type': 'integer'}},
  'required': ['num1', 'num2'],
  'title': 'Input for `abc`',
  'type': 'object'}}

In [6]:
model_name = os.getenv("FIREWORKS_MODEL_NAME")

**Setup LLM**

In [7]:
llm = ChatFireworks(
    model=model_name,
    temperature=os.getenv("TEMPERATURE"),               # Lower temp for accuracy
    max_tokens=200,                # Limit output to save cost
    model_kwargs={"top_p": 0.9},                     # Reasonable diversity
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="compute abc between 2 and 3")
]
response = llm.invoke(messages)

/tmp/ipykernel_18014/1514655232.py:1: LangChainDeprecationWarning: The class `ChatFireworks` was deprecated in LangChain 0.0.26 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-fireworks package and should be used instead. To use it run `pip install -U :class:`~langchain-fireworks` and import as `from :class:`~langchain_fireworks import ChatFireworks``.
  llm = ChatFireworks(
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x71779b9a5550>


In [8]:
print(response.content)

<think>
Okay, the user asked to "compute abc between 2 and 3." Hmm, first I need to figure out what exactly they mean. The term "abc" isn't a standard mathematical function or operation that I know of. Maybe they meant something else. Let me think.

Could "abc" be a typo or shorthand for something else? Like, maybe they meant "a times b times c" where a, b, c are numbers between 2 and 3? But that's not clear. Alternatively, maybe it's a reference to a specific formula or problem. Wait, sometimes in math problems, especially in algebra or calculus, variables like a, b, c can represent coefficients or constants. But without more context, it's hard to tell.

Another possibility is that the user is referring to the abc conjecture, which is a famous problem in number theory. The abc conjecture involves triples of positive integers a, b, c such that a + b = c and they are coprime. But the user mentioned "between 2 and 3," which might not fit directly. The abc conjecture isn't about computing

In [9]:
from fireworks.client import ChatCompletion

messages= [
    {"role": "user", "content": "Compute abc between 2 and 3"}
]

# Pass th function to LLM model
response = ChatCompletion.create(
    model=model_name,
    messages=messages,
    functions=[abc_json],
    function_call="auto",
    temperature=0
)

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x71779b9a6fc0>


In [10]:
response

ChatCompletionResponse(id='857acfbf-a6cb-4482-9b37-82648bc2edc9', object='chat.completion', created=1754126005, model='accounts/fireworks/models/qwen3-30b-a3b', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content="<think>\nOkay, the user wants me to compute abc between 2 and 3. Let me check the tools provided. There's a function called abc with parameters num1 and num2, both integers. The required parameters are num1 and num2. So I need to call this function with num1 as 2 and num2 as 3. I should structure the tool call in JSON inside the specified XML tags. Let me make sure the JSON is correctly formatted with the right parameters.\n</think>\n\n", reasoning_content=None, tool_calls=[ChatCompletionMessageToolCall(index=0, id='call_5Sxq29N2KyVMr2r7Jyomp0g7', type='function', function=ChatCompletionMessageToolCallFunction(name='abc', arguments='{"num1": 2, "num2": 3}'))], tool_call_id=None, function=None, name=None), finish_reason='tool_calls', lo

**Executing the function by extracting the info from the output of the model**

In [11]:
print(response.choices[0].message.tool_calls[0].function)
print(response.choices[0].message.tool_calls[0].function.arguments)
print(type(response.choices[0].message.tool_calls[0].function.arguments))

name='abc' arguments='{"num1": 2, "num2": 3}'
{"num1": 2, "num2": 3}
<class 'str'>


In [12]:
func_name = response.choices[0].message.tool_calls[0].function.name
func_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)

print("Function name:", func_name)
print("Function arguments:", func_args)
print(type(func_args))

Function name: abc
Function arguments: {'num1': 2, 'num2': 3}
<class 'dict'>


In [13]:
if func_name == 'abc':
    result = abc(**func_args)
print(result)

-2


## **Using Langchain**

In [14]:
from langchain_core.tools import tool

@tool
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [15]:
abc.description

'Compute abc between two numbers'

In [16]:
from typing import Callable
def jsonschema(f: Callable) -> dict:
    kw = {
        n: (o.annotation, ... if o.default == inspect.Parameter.empty else o.default)
        for n, o in inspect.signature(f).parameters.items()
    }
    schema = create_model(f"Input for {f.__name__}", **kw).schema()
    return {
        "name": f.__name__,
        "description": f.__doc__,
        "parameters": schema
    }

# Get JSONSchema of function
abc_json = jsonschema(abc.func)
tools = [abc_json]

response = ChatCompletion.create(
    model=model_name,
    messages=messages,
    functions=tools,
    function_call="auto",
    temperature=0
)

/tmp/ipykernel_18014/1970905835.py:7: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  schema = create_model(f"Input for {f.__name__}", **kw).schema()
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x71779b9ce300>


In [17]:
response.choices[0].message.tool_calls[0]

ChatCompletionMessageToolCall(index=0, id='call_YXqXqEtxXR136mjUmFVprykV', type='function', function=ChatCompletionMessageToolCallFunction(name='abc', arguments='{"num1": 2, "num2": 3}'))